In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import json

In [2]:
df_train = pd.read_csv('../data/processed/train.csv', delimiter=',')
df_test = pd.read_csv('../data/processed/test.csv', delimiter=',')
df_train.head()

C:\Users\JShah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:

def get_max_qid(df):
    max_qid = 0
    for idx,frame in df.iterrows():
        qid1 = int(frame['qid1'])
        qid2 = int(frame['qid2'])
        if qid1>max_qid:
            max_qid = qid1
        elif qid2>max_qid:
            max_qid = qid2
    print('Max qid = ', max_qid)
    return max_qid

max_qid = get_max_qid(df_train)

Max qid =  537932


In [4]:
def group_questions(df):
    group_id = 0
    group_list = np.repeat(-1, max_qid)
    
    for idx,frame in df.iterrows():
        qid1 = int(frame['qid1'])
        qid2 = int(frame['qid2'])
        
        if int(frame['is_duplicate'])==1:
            # if both has no group, add new group
            if group_list[qid1]==-1 and group_list[qid2]==-1:
                group_list[qid1] = group_id
                group_list[qid2] = group_id
                group_id += 1

            # if both has group, join the group 
            elif group_list[qid1]!=-1 and group_list[qid2]!=-1 :
                idxes_to_be_joined = np.where(group_list==group_list[qid2])[0]
                group_list[idxes_to_be_joined] = group_list[qid1]

            # only q1 has group , than add q2 to q1's group
            elif  group_list[qid1]!=-1:
                group_list[qid2] = group_list[qid1]

            # only q2 has group , than add q1 to q2's group
            elif  group_list[qid2]!=-1:
                group_list[qid1] = group_list[qid2]
                
    return group_list
    
group_ids = group_questions(df_train)

In [5]:
sum(group_ids!=-1) 

149650

In [8]:
print(group_ids[group_ids != -1])

[    0     0     1 ... 64024 64025 64025]


In [9]:
# Get all the group and store it as a dictionary
group_dict = {}
for i in range(np.max(group_ids)+1):
    group_members = np.where(group_ids==i)[0]
    if len(group_members)>0:
        group_dict[i] = group_members

In [21]:
import itertools

def enumerate_all_positive_cases(group_dict):
    
    def enumerate_inside_group(group):
        return list(itertools.combinations(group, 2))
    
    return np.vstack(enumerate_inside_group(group_dict[group_id]) for group_id in group_dict)

def duplicate_all(df):
    
    def get_qid_set():
        ids = set()
        for i,series in df.iterrows():
            if series['qid1'] not in ids:
                ids.add(series['qid1'])
            if series['qid2'] not in ids:
                ids.add(series['qid2'])
        return ids
    
    id_set = get_qid_set()
    return [[i,i] for i in id_set]

In [22]:
# Enumerate all cases of duplicated question pairs from each group
enumerate_pairs = enumerate_all_positive_cases(group_dict)

# The question pairs with itself is also a sample of duplicated question pair
# duplicate_pairs = duplicate_all(df_train) 

all_pos_pairs = enumerate_pairs
# all_pos_pairs = np.vstack([enumerate_pairs,duplicate_pairs])

C:\Users\JShah\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


In [23]:
print('The duplicate question pair count grows from {} to {}'.format(len(df_train[df_train['is_duplicate']==1]),len(all_pos_pairs))) # The total duplicated samples count

The duplicate question pair count grows from 149263 to 228548


In [24]:
def get_pos_rate_in_training_set():
    dup = np.array(df_train['is_duplicate'])
    pos_ratio = np.sum(dup) / dup.shape[0]
    return pos_ratio

In [25]:
# since we are very probable to use a single question for several times, we should remove validation samples directly at this point
# Fro example, Q_a == Q_b and Q_b == Q_c ,
# In my method, we'll generate a new data Q_a == Q_c
# If we move this Q_a == Q_c sample to validation set,
#     it is very weird that our training set already has this kind of information (can be recognized from the Q_a == Q_b == Q_c relation).

import random

validation_size = 20000 # an approximation, final result can be slightly more than this number

def split_val(pos_pairs, val_pos_ratio):
    
    # an estimation of how many data should be split from training set
    split_pos_size = int(validation_size * val_pos_ratio)
    
    qids = pos_pairs.flatten()
    
    # totally remove those selected qids from training set
    val_bools = np.repeat(False,pos_pairs.shape[0])
    while(np.sum(val_bools)<split_pos_size):
        rnd_qid = random.randint(0,len(qids)-1)
        val_single_bool = np.bitwise_or(pos_pairs[:,0]==rnd_qid,pos_pairs[:,1]==rnd_qid)
        val_bools = np.bitwise_or(val_bools, val_single_bool)
    val_idxes = np.where(val_bools)[0]
    
    val = pos_pairs[val_idxes]
    train = np.delete(pos_pairs, val_idxes, axis=0)
    
    return train, val

In [26]:
val_pos_ratio = get_pos_rate_in_training_set()
train_pos_pairs, val_pos_pairs = split_val(all_pos_pairs, val_pos_ratio)

In [27]:
print('Training positive pairs:', train_pos_pairs.shape[0])
print('Validation positive pairs:', val_pos_pairs.shape[0])

Training positive pairs: 221158
Validation positive pairs: 7390


In [29]:
pickle.dump(train_pos_pairs, open('../data/processed/train_positive_qid_pairs.pkl', 'wb'))
pickle.dump(val_pos_pairs, open('../data/processed/validation_positive_qid_pairs.pkl', 'wb'))

In [30]:
non_duplicate = df_train[df_train['is_duplicate']==0]
non_dup_question_pairs = np.array([[series['qid1'],series['qid2']] for i,series in non_duplicate.iterrows()])

val_pos_count = val_pos_pairs.shape[0]
val_total_count = val_pos_count / val_pos_ratio
val_neg_count = int(val_total_count - val_pos_count)

val_neg_idxes = [random.randint(0,len(non_dup_question_pairs)-1) for i in range(val_neg_count)]

val_neg_pairs = non_dup_question_pairs[val_neg_idxes]
train_neg_pairs = np.delete(non_dup_question_pairs, val_neg_idxes, axis=0)

In [31]:
print('Gauranteed non-duplicated question pair length is ', len(non_dup_question_pairs), '\n')

print('Training negative pairs:', train_neg_pairs.shape[0])
print('Validation negative pairs:', val_neg_pairs.shape[0])

Gauranteed non-duplicated question pair length is  255027 

Training negative pairs: 242705
Validation negative pairs: 12626


In [32]:

pickle.dump(train_pos_pairs, open('../data/processed/train_negative_qid_pairs.pkl', 'wb'))
pickle.dump(val_pos_pairs, open('../data/processed/validation_negative_qid_pairs.pkl', 'wb'))

In [33]:
print('Validation final size:', val_neg_pairs.shape[0] + val_pos_pairs.shape[0] )

Validation final size: 20016


In [35]:

df_train = pd.read_csv('../data/processed/train.csv', delimiter=',')
# enc_map = pickle.load(open('../data/processed/enc_map.pkl','rb'))

In [36]:
import re

def parse_wrod_list(question):
    
    if type(question)!=str:
        return []
    
    # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
    special_chars = '[\s\'!"\?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]'
    pre_separator = '(?='+special_chars+')'
    post_separator = '(?='+special_chars+'|$)'
    single_word = '[^\s\-]+' # non-empty is enough here

    return re.findall(special_chars+single_word+post_separator, question)

In [37]:
def gen_qid_question_dict(df, parse=False):
    res = {}
    for i,frame in df.iterrows():
        
        qid1 = int(frame['qid1'])
        if qid1 not in res:
            if parse:
                res[qid1] = parse_wrod_list(frame['question1'])
            else:
                res[qid1] = frame['question1']
        
        qid2 = int(frame['qid2'])
        if qid2 not in res:
            if parse:
                res[qid2] = parse_wrod_list(frame['question2'])
            else:
                res[qid2] = frame['question2']
            
    return res

In [38]:

training_question_dict = gen_qid_question_dict(df_train)

In [39]:
pickle.dump(training_question_dict, open('../data/processed/qid_question_dict.pkl', 'wb'))